In [3]:
from tenacity import (retry, stop_after_attempt, wait_random_exponential)
import random
import pandas as pd
import openai
import gsearch
import json
import os
import tiktoken
import time
import inspect
import callgpt
import react_chain
import compliance_checker
import excel_analysis
import agents_report
import agents_discussion
import repo_reader
import read_file
import re
import tkinter as tk
from tkinter import ttk
from openpyxl import load_workbook
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from tkinter import filedialog
from tkinter import *
from collections import deque
from dotenv import load_dotenv

def count_tokens(text):
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(text))
    return num_tokens

load_dotenv() 
callgpt = callgpt.Ask()
current_dir = os.getcwd()
os.chdir(current_dir)

class BusinessAnalyst:
    task_list = []
    def __init__(self):
        self.role_system = {"role": "system", "content": "You are a highly organised and detail oriented Goldman Sachs business analyst."}
        self.messages = [self.role_system]
        self.task_list = deque()
        # INSERT INFO ABOUT SAVING THE PROMPT + COMPLETION PAIRS - FAISS? CHROMA?

    def write_output_to_file(self, output, file_name='Outputs/analyst_output.txt', mode='a'):
        with open(file_name, mode, encoding='utf-8') as f:
            f.write(f"{output} \n")

    @retry(wait=wait_random_exponential(min=2, max=20), stop=stop_after_attempt(3))
    def chat_with_gpt3(self, prompt, max_retries=3, delay=2):
        user_message = {"role": "user", "content": prompt}
        new_message_tokens = count_tokens(prompt)
        total_tokens = sum(count_tokens(m["content"])
                        for m in self.messages) + new_message_tokens
        while total_tokens + 1 > 3500:
            if len(self.messages) >= 3:
                self.messages.pop(1)
                self.messages.pop(1)
                total_tokens = sum(count_tokens(m["content"])
                                for m in self.messages) + new_message_tokens
            else:
                print("Reached the token limit, please reduce the length of your input.")
                break

        self.messages.append(user_message)
        response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=self.messages
                )
        assistant_message = {
            "role": "assistant", "content": response.choices[0].message["content"]}
        self.messages.append(assistant_message)
        return assistant_message["content"]

    def process_input(self, user_query):
        prompt = f"You are an AI who wants to answer the following user query: '{user_query}' with a numbered list of tasks. Response: "
        processed_query = self.chat_with_gpt3(prompt)
        tasks = processed_query.split("\n")
        self.task_list.extend(task.strip() for task in tasks if task.strip())
        print("\033[92m\033[1m"+"\n*****Processed query:*****\n" + "\033[0m\033[0m"+processed_query)
        self.write_output_to_file(processed_query)
        
    # Not strictly speaking necessary, unless we want to do nested tasks
    def decompose_tasks(self): 
        task = self.task_list.popleft()
        prompt = f"You are an AI whose job is to assess the following processed query into the most feasible first task to start with and create a new numbered task list: '{task}'. Response: "
        sub_tasks = self.chat_with_gpt3(prompt)
        tasks = sub_tasks.split("\n")
        self.task_list.extend(task.strip() for task in tasks if task.strip())
        print("\033[92m\033[1m"+"\n*****Subtasks*****\n"+"\033[0m\033[0m" + sub_tasks)
        self.write_output_to_file(sub_tasks)
      
    def select_task(self):
        # Implement your logic for selecting the task here
        selected_task_index = 0
        task = self.task_list.pop(selected_task_index)
        return task
    
    @retry(wait=wait_random_exponential(min=2, max=20), stop=stop_after_attempt(3), reraise=True)
    def choose_best_tool(self, task):
        prompt = f"""You are an AI that has to select the best tool to use to solve the following task: '{task}'.
        Please respond in the format 'Action: [tool_name] only including items from this list - Action: (gsearch|repo_reader|ask_user_input|agent_discussion|agent_report|analyse_excel.
        This is how you make your choice:
        If you think talking it through might help: agent_discussion
        If you want to write a report: agent_report
        If you need to seach the web: gsearch
        If you need user input: ask_user_input
        The response should contain absolutely no other words. 
        The default option should be 'Action: gsearch' """
        response = self.chat_with_gpt3(prompt)
        tool_name = response.split(': ')[1]  # Split the response and retrieve the tool name

        print(f"GPT-3 Response: {response}")
        self.write_output_to_file(f"GPT-3 Response: {response}")  
        tool_re = re.compile(r"'\s*(.*?)\s*'")
        tool_match = tool_re.search(response)
        if not tool_match:
            tool_re = re.compile(r"Action:\s*(gsearch|repo_reader|ask_user_input|agent_discussion|agent_report|analyse_excel)", re.IGNORECASE)
            tool_match = tool_re.search(response)

        if tool_match:
            best_tool = tool_match.group(1)
            print(f"Extracted tool from GPT-3 response: {best_tool}")
            self.write_output_to_file(f"Extracted tool from GPT-3 response: {best_tool}")
            return best_tool.strip().lower().replace(" ", "_")
        else:
            print(f"Unexpected GPT-3 response: {response}")
            self.write_output_to_file(f"Unexpected GPT-3 response: {response}")
            return self.user_select_tool()

    def user_select_tool(self):
        print("Unexpected GPT-3 response. Please select the next task to perform:")
        tools = ["gsearch", "repo_reader", "ask_user_input", "agent_discussion", "agent_report", "analyse_excel"]

        for i, tool in enumerate(tools, start=1):
            print(f"{i}. {tool}")
            self.write_output_to_file(f"Unexpected GPT-3 response. Please select the next task to perform: {i}. {tool}")

        while True:
            try:
                user_choice = int(input("Enter the number of your choice: "))
                if 1 <= user_choice <= len(tools):
                    return tools[user_choice - 1]
                else:
                    print("Invalid choice. Please enter a number between 1 and 6.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    def ask_user_input(self, prompt):
        user_input = input("What should I do\n" + prompt)
        self.write_output_to_file(f"What should I do re {prompt} and {user_input}")
        result = self.chat_with_gpt3(f"re {prompt} I think we should do {user_input}")
        return result

    def agent_discussion(self, prompt):
        discussion = agents_discussion.generate_output(prompt)
        self.write_output_to_file(discussion)
        return discussion

    def agent_report(self, prompt):
        report = agents_report.create_joint_letter(prompt)
        self.write_output_to_file(report)
        return report
    
    @retry(wait=wait_random_exponential(min=2, max=20), stop=stop_after_attempt(3))
    def execute_tasks(self):
        results = []
        task = self.task_list.popleft()
        function_mapping = {
            "gsearch": gsearch.execute,
            "repo_reader": repo_reader.main,
            "analyse_excel": self.analyse_excel,
            "ask_user_input": self.ask_user_input,
            "agent_discussion": self.agent_discussion,
            "agent_report": self.agent_report
        }

        best_tool = self.choose_best_tool(task)
        if best_tool in function_mapping:
            tool_function = function_mapping[best_tool]
            print(f"Recommended tool is {best_tool}: {tool_function}")

            # Ask for user confirmation or alternative selection
            user_confirm = input(f"Do you want to proceed with the recommended tool '{best_tool}'? (Y/N) or choose another tool from the list {list(function_mapping.keys())}: ")

            if user_confirm.lower() == 'y':
                result = tool_function(task)  # Use the selected function to solve the task

            elif user_confirm.lower() == 'n':
                print("User decided not to proceed with the recommended tool.")

            elif user_confirm.lower() in function_mapping.keys():
                chosen_tool = user_confirm.lower()
                print(f"User chose to proceed with the tool: {chosen_tool}")
                result = function_mapping[chosen_tool](task)  # Use the user-selected function to solve the task

            else:
                print("Invalid input. Retrying...")
                return None  # Retry the task

            print(f"\033[92m\033[1m \n*****Results Of Execution*****\n \033[0m\033[0m {result}")
            self.write_output_to_file(f"\n*****Results Of Execution*****\n {result}")
            return result
        else:
            print(f"Invalid tool selected: {best_tool}. Retrying...")


    def refine_tasks(self):
        prompt = f"Considering the results, let's refine and create a new list of tasks based on the following analysis: {self.task_list}. The response should be in a numbered list with one sentence each. Response: "
        new_tasks = self.chat_with_gpt3(prompt)
        tasks = new_tasks.split("\n")
        self.task_list.extend(tasks)
        print("\033[92m\033[1m"+"\n*****Refined New Tasks*****\n"+"\033[0m\033[0m"+ (new_tasks))
        self.write_output_to_file("\n*****Refined New Tasks*****\n" + new_tasks)

    def analyze_results(self, results):
        prompt = f"You are an AI focused on analysing the results from executing the task: '{results}'. Please critically explain and appraise the output. Answer what are avenues to further explore and understand from this. "
        analysis = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Analysis*****\n"+"\033[0m\033[0m" + (analysis))
        self.write_output_to_file("\n*****Analysis*****\n" + analysis)
        return analysis
        
    def generate_output(self, analysis):
        prompt = f"You are an analyst who will create a readable document from the whole discussion summarising it and clearly articulating key theories, ideas and actions to be done. Please generate a user-friendly output based on the following analysis: {analysis}. Response: "
        output = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Generated Output*****\n"+"\033[0m\033[0m"+ (output))
        self.write_output_to_file("\n*****Generated Output*****\n" + output)
        return output

    def run_analysis(self, user_query):
        self.process_input(user_query)
        self.decompose_tasks()
        results = self.execute_tasks()
        self.refine_tasks()
        ques = input("Do you want to continue - Y or N? ")
        while ques == "Y":
            self.process_input(user_query)
            self.decompose_tasks()
            results = self.execute_tasks()
            self.refine_tasks()
            ques = input("Do you want to continue - Y or N? ")
        if ques == "N":
            print("\033[92m\033[1m" + "\n*****Alright! Final Results Coming Up*****\n" + "\033[0m\033[0m")
            self.write_output_to_file("\n*****Alright! Final Results Coming Up*****\n")
        else:
            print("\n\n*******Dude! I said Y or N!*******\n\n")
        analysis = self.analyze_results(results)
        output = self.generate_output(analysis)
        return output

    def get_excel_file_path(self):
        root = Tk()
        root.withdraw()
        file_path = filedialog.askopenfilename(initialdir="/", title="Select a File", filetypes=[("Excel Files", "*.xlsx")])
        print(file_path)
        root.destroy()
        return file_path

    def analyse_excel(self, question):
        print("\033[92m\033[1m" + "\n*****Excel Analysis*****\n" + "\033[0m\033[0m")
        self.write_output_to_file("\n*****Excel Analysis*****\n")
        filepath = self.get_excel_file_path()
        results = excel_analysis.main(filepath)
        df = pd.read_excel(filepath)
        column_names = df.columns.tolist()
        user_help = input("Please provide more information on how to analyze the Excel file: ")
        prompt = f"With the user's input: '{user_help}', analyze the Excel file with '{column_names}' as columns whose content summary is '{results}' and determine what operations you need to do to figure out how you would answer '{question}'."

        gpt3_response = self.chat_with_gpt3(prompt)
        print(f"GPT-3 Response: {gpt3_response}")

        prompt = f"As a top analyst at Goldman Sachs, when presented with an Excel sheet with a company's financial statements, define what tests and hypotheses would you employ to determine if the company is investable? Write your response only as a numbered task list: "
        task_list = self.chat_with_gpt3(prompt)
        # xlwings to analyse excel files?
        sub_tasks = self.process_input(f"Given this, how would you go about answering Question:{prompt}. This is the Output re the excel structure:{task_list}")
        tasktodo = self.select_task(sub_tasks)
        print(f"\n**My First Task Is: {tasktodo}**")
        self.write_output_to_file(f"\n**My First Task Is: {tasktodo}**\n")
        result = self.execute_tasks(tasktodo)
        
        return result

ba = BusinessAnalyst()
result = None

def run():
    # Edit this to call ba.choose_best_tool to choose
    while True:
        try:
            user_query = input("""Choose one. What would you like to do? \n
                        1. Ask something you're curious about, run tasks \n
                        2. Figure out something about your docs \n
                        3. Figure out what's inside a python repo\n
                        4. Simulate a dialogue between two people \n
                        5. Let's write a report together! \n
                        6. Analyse an excel! \n
                        7. Read a document and return text. \n
                        00. Exit the prompt \n
                        """)
            if user_query == "1":
                prompt = input("Enter your query: ")
                print(
                    f"""\033[92m\033[1m" + "\n*****Your Question*****\n" + "\033[0m\033[0m {prompt}""")
                ba.write_output_to_file(
                    f"\n*****Your Question*****\n is {prompt}")
                result = ba.run_analysis(prompt)
            elif user_query == "2":
                # Call the function to figure out something about your docs
                # Replace this line with the appropriate function call
                result = "\n\nFigure out something about your docs (function not implemented)\n\n"
            elif user_query == "3":
                # Call the function to figure out what's inside a python repo
                # Replace this line with the appropriate function call
                result = "\n\nFigure out what's inside a python repo (function not implemented)\n\n"
            elif user_query == "4":
                # Call the function to simulate a dialogue between two people
                import agents_discussion
                result = agents_discussion.generate_output(user_query)
                print("\n\nHere's a dialogue!\n\n")
            elif user_query == "5":
                # Call the function to write a report together
                import agents_report
                result = agents_report.create_joint_letter(user_query)
                print("\n\nHere's the report!\n\n")
            elif user_query == "6":
                # Call the function to analyze an excel
                result = ba.analyse_excel("Analyze an excel sheet")
            elif user_query == "7":
                fp = filedialog.askopenfilename(
                    initialdir="/", title="Select a File")
                result = read_file.read_file(fp)
                ba.run_analysis(res1ult)
            elif user_query == "00":
                result = "\n\nSorry to see you go !!\n\n"
                exit()
            else:
                print("\n\nInvalid option. Please choose a number between 1 and 7.\n\n")
            if result:
                print(result)
                ba.write_output_to_file(result)
        except TypeError as e:
            print(f"An error occurred: {e}\nPlease try again.")
        except ValueError as e:
            print(f"An error occurred: {e}\nPlease try again.")

In [4]:
run()

" + "
*****Your Question*****
" + " plan a trip to rome

*****Processed query:*****
Sure, I can help you plan a trip to Rome! Here is a list of tasks to get you started:

1. Research flights and book airfare
2. Reserve accommodations in Rome, such as a hotel or Airbnb
3. Create an itinerary of attractions and activities you want to visit
4. Purchase tickets or make reservations in advance for popular attractions, such as the Colosseum, Vatican Museums, and Sistine Chapel
5. Plan out transportation options, such as public transportation, taxi services, or car rentals
6. Look up local restaurants and make dinner reservations
7. Exchange currency or obtain international credit card if necessary
8. Pack essentials for your trip, such as comfortable walking shoes, clothing suitable for the weather, and any necessary travel accessories. 

These are just some starting points for your planning process. Good luck, and enjoy your trip to Rome!

*****Subtasks*****
Based on the current list provid

INFO:openai:error_code=None error_message='That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID c24b1ba9a75c4aa895d736413a68af41 in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False


GPT-3 Response: Action: gsearch
Extracted tool from GPT-3 response: gsearch
Recommended tool is gsearch: <function execute at 0x16a436b80>
User decided not to proceed with the recommended tool.
GPT-3 Response: Action: gsearch
Extracted tool from GPT-3 response: gsearch
Recommended tool is gsearch: <function execute at 0x16a436b80>
User chose to proceed with the tool: agent_discussion

A_GPT-6
1. Brainstorm a list of attractions and activities that you are interested in visiting. 
2. Research each attraction and activity to determine their location, hours of operation, admission fees, and any other relevant details. 
3. Prioritize your list of attractions and activities based on your personal interest and time constraints. 
4. Determine the most efficient route or schedule for visiting each attraction or activity. 
5. Make reservations or purchase tickets in advance, if necessary, to avoid any potential delays or cancellations. 
6. Create a comprehensive itinerary that includes the date

KeyboardInterrupt: Interrupted by user

: 